# RAPIDS & Scanpy Single-Cell RNA-seq Multi-GPU Workflow on 1 Million Cells

Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License") you may not use this file except in compliance with the License. You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This notebook demonstrates a single-cell RNA analysis workflow that begins with preprocessing a count matrix of size `(n_gene, n_cell)` and results in a visualization of the clustered cells for further analysis.

For demonstration purposes, we use a dataset of 1M brain cells with Unified Virtual Memory to oversubscribe GPU memory. We then use dask to scale PCA, K-means clustering, and UMAP across multiple GPUs.

See the README for instructions to download this dataset.

## Import requirements

In [1]:
import time
import pandas
import numpy as np
import scanpy as sc
import anndata

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import cupy as cp
import cupyx as cpx
import cudf
import math

import h5py
import scipy

import dask
import dask_cudf
import rmm

from numba import cuda

from dask_cuda import initialize, LocalCUDACluster
from dask import delayed, dataframe
from dask.dataframe.utils import make_meta
from dask.distributed import Client

from cuml.manifold import TSNE, UMAP

from cuml.dask.decomposition import PCA
from cuml.dask.cluster import KMeans
from cuml.dask.manifold import UMAP as distUMAP
from cuml.dask.linear_model import LinearRegression as distLR

import rapids_scanpy_funcs
import utils

import warnings
warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

We use the RAPIDS memory manager to enable Unified Virtual Memory management, which allows us to oversubscribe the GPU memory

In [2]:
cluster_start = time.time()

enable_tcp_over_ucx = True
enable_nvlink = True
enable_infiniband = False

rmm.reinitialize(managed_memory=True)
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

initialize.initialize(create_cuda_context=True,
                      enable_tcp_over_ucx=enable_tcp_over_ucx,
                      enable_nvlink=enable_nvlink,
                      enable_infiniband=enable_infiniband)

cluster = LocalCUDACluster(protocol="ucx",
                           CUDA_VISIBLE_DEVICES=[0, 1],
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)

client = Client(cluster)
n_workers = len(client.scheduler_info()['workers'].keys())
print(n_workers)
client

2


Client Scheduler: ucx://127.0.0.1:60198 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 33.58 GB


## Input data

In the cell below, we provide the path to the sparse `.h5ad` file containing the count matrix to analyze. Please see the README for instructions on how to download the dataset we use here.

To run this notebook using your own dataset, please see the README for instructions to convert your own count matrix into this format. Then, replace the path in the cell below with the path to your generated `.h5ad` file.

In [3]:
import os, wget
# input_file = "../1M_neurons_filtered_gene_bc_matrices_h5.h5"
# input_file = "../data/1M_neurons_filtered_gene_bc_matrices_h5.h5"
input_file = "../data/1M_brain_cells_10X.sparse.h5ad"

if not os.path.exists(input_file):
    print('Downloading import file...')
    os.makedirs('../data', exist_ok=True)
    wget.download(
        'https://rapids-single-cell-examples.s3.us-east-2.amazonaws.com/1M_brain_cells_10X.sparse.h5ad',
        input_file)

## Set parameters

In [4]:
# marker genes
MITO_GENE_PREFIX = "mt-"              # Prefix for mitochondrial genes to regress out
markers = ["Stmn2", "Hes1", "Olig1"]  # Marker genes for visualization

# filtering cells
min_genes_per_cell = 200      # Filter out cells with fewer genes than this expressed 
max_genes_per_cell = 6000     # Filter out cells with more genes than this expressed 
min_cells=1                   # Genes containing a number of cells below this value will be filtered

# filtering genesinitialize
n_top_genes = 4000            # Number of highly variable genes to retain

# PCA
n_components = 50             # Number of principal components to compute
pca_train_ratio = 0.35        # percentage of cells to use for PCA training
n_pca_batches = 10

# t-SNE
tsne_n_pcs = 20               # Number of principal components to use for t-SNE

# k-means
k = 35                        # Number of clusters for k-means

# KNN
n_neighbors = 15              # Number of nearest neighbors for KNN graph
knn_n_pcs = 50                # Number of principal components to use for finding nearest neighbors

# UMAP
umap_train_ratio = 0.35
umap_min_dist = 0.3
umap_spread = 1.0

start = time.time()

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [5]:
data_load_start = time.time()

In [6]:
# TODO: Compute batches using total Rows and columns
BATCHES = n_workers * 3

# Read along with filtering
with h5py.File(input_file, 'r') as h5f:
    indices = h5f['/X/indptr']
    data = h5f['/X/data']
    orginal_genes = h5f['/var/_index']
    
    orginal_genes = cudf.Series(orginal_genes, dtype=cp.dtype('object'))

    total_cols = orginal_genes.shape[0]
    total_rows = indices.shape[0] - 1
    batch_rows = math.ceil(total_rows / BATCHES)

In [7]:
%%time

# TODO: Compute batches using total Rows and columns
BATCHES = n_workers * 3
    
@delayed
def read_partition(sample_file, ds_data, ds_indices, ds_indptr, 
                   batch, rows, total_cols, gene_filter,
                   min_genes_per_cell=200, max_genes_per_cell=6000, target_sum=1e4):
    """
    Loads a single partition from HDF5 file.
    """
    batch_start = batch * rows
    batch_end = batch_start + rows

    with h5py.File(input_file, 'r') as h5f:
        # Read all things row pointers for one worker
        indptrs = h5f[ds_indptr]
        start_ptr = indptrs[batch_start]
        end_ptr = indptrs[batch_end]

        # Read all things data for one worker
        data = h5f[ds_data]
        sub_data = cp.array(data[start_ptr:end_ptr])

        # Read all things column pointers for one worker
        indices = h5f[ds_indices]
        sub_indices = cp.array(indices[start_ptr:end_ptr])

        # recompute the row pointer for the partial dataset
        sub_indptrs  = cp.array(indptrs[batch_start:(batch_end + 1)])
        first_ptr = sub_indptrs[0]
        sub_indptrs = sub_indptrs - first_ptr

    partial_sparse_array = cp.sparse.csr_matrix((sub_data, sub_indices, sub_indptrs),
                                                shape=(batch_end - batch_start, total_cols))
    # TODO: Add barcode filtering here.
    degrees = cp.diff(partial_sparse_array.indptr)
    query = ((min_genes_per_cell <= degrees) & (degrees <= max_genes_per_cell))
    partial_sparse_array = partial_sparse_array[query]

    if not gene_filter:
        ret_value = partial_sparse_array.sum(axis=0)
    else:
        partial_sparse_array = rapids_scanpy_funcs.normalize_total(
            partial_sparse_array, target_sum=target_sum)
        ret_value = partial_sparse_array[:, gene_filter]

    return ret_value

CPU times: user 0 ns, sys: 112 µs, total: 112 µs
Wall time: 117 µs


In [8]:
%%time
print('Computing sum and creating gene count filter...')
dls_sum = []
for batch in range(BATCHES):
    rows = min(batch_rows, total_rows - (batch_rows * batch))
    if batch < 4:
        dls_sum.append(
            dask.array.from_delayed(
                read_partition(input_file, 
                               '/X/data', '/X/indices', '/X/indptr', 
                               batch, rows, total_cols, None,
                               min_genes_per_cell=min_genes_per_cell,
                               max_genes_per_cell=max_genes_per_cell),
                dtype=cp.float32,
                shape=(rows, total_cols)))

sum_gpu_array = dask.array.concatenate(dls_sum)
sum_gpu_array = sum_gpu_array.compute()

# Filter genes with at-least <<min_cells>> number of cells rec.
sum_gpu_array = (sum_gpu_array.sum(axis=0) >= min_cells)

Computing sum and creating gene count filter...


ValueError: unsupported indexing

In [9]:
%%time
print('Loading data into dataframes...')

dls_data = []
for batch in range(BATCHES):
    rows = min(batch_rows, total_rows - (batch_rows * batch))
    if batch < 4:
        dls_data.append(
        dask.array.from_delayed(
            read_partition(input_file, 
                           '/X/data', '/X/indices', '/X/indptr', 
                           batch, rows, total_cols, sum_gpu_array.tolist(),
                           min_genes_per_cell=min_genes_per_cell,
                           max_genes_per_cell=max_genes_per_cell),
            dtype=cp.float32,
            shape=(rows, total_cols)))

print('Concate sub-arrays...')
sparse_gpu_array = dask.array.concatenate(dls_data)

print('Compute and persist arrays...')
type(sparse_gpu_array)
sparse_gpu_array = sparse_gpu_array.persist()
sparse_gpu_array.shape

Loading data into dataframes...


AttributeError: 'Array' object has no attribute 'tolist'

For this example, we select the first 1 million cells in the dataset. We maintain the index of unique genes in our dataset:

Verify the shape of the resulting sparse matrix:

In [10]:
%%time
sparse_gpu_array = dask.array.log1p(sparse_gpu_array)

NameError: name 'sparse_gpu_array' is not defined

And the number of non-zero values in the matrix:

In [11]:
%%time
sparse_gpu_array = sparse_gpu_array.compute()

NameError: name 'sparse_gpu_array' is not defined

In [12]:
print("Total data load and format time: %s" % (time.time() - data_load_start))

Total data load and format time: 298.79897594451904


## Preprocessing

In [13]:
preprocess_start = time.time()

### Filter

In [14]:
# Filter out the genes that got removed from the original data
genes = orginal_genes[sum_gpu_array]
genes.shape

IndexError: Failed to convert index to appropirate row

We filter the count matrix to remove cells with an extreme number of genes expressed.

In [ ]:
%%time
# FILTERING IS NOT DONE WHILE READING DATA
# sparse_gpu_array = rapids_scanpy_funcs.filter_cells(sparse_gpu_array, 
#                                                     min_genes=min_genes_per_cell, 
#                                                     max_genes=max_genes_per_cell)

Some genes will now have zero expression in all cells. We filter out such genes.

In [ ]:
%%time
# FILTERING IS NOT DONE WHILE READING DATA
# sparse_gpu_array, genes = rapids_scanpy_funcs.filter_genes(sparse_gpu_array, genes.get(), min_cells=1)

The size of our count matrix is now reduced.

In [ ]:
%%time
# sparse_gpu_array.shape

### Normalize

We normalize the count matrix so that the total counts in each cell sum to 1e4.

In [ ]:
%%time
# sparse_gpu_array = rapids_scanpy_funcs.normalize_total(sparse_gpu_array, target_sum=1e4)
# sparse_gpu_array = sparse_gpu_array.log1p()

sparse_gpu_array = sparse_gpu_array.astype(cp.float32)
sparse_gpu_array.shape

Next, we log transform the count matrix.

In [ ]:
%%time
marker_genes_raw = sparse_gpu_array[:, genes[genes.isin(markers)].index.to_arrow().to_pylist()]

### Select Most Variable Genes

We convert the count matrix to an annData object.

In [ ]:
%%time
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = genes.to_pandas()

Before filtering the count matrix, we save the 'raw' expression values of the marker genes to use for labeling cells afterward.

Using scanpy, we filter the count matrix to retain only the most variable genes.

In [ ]:
%%time
sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor="cell_ranger")
print("Full time: %s" % (time.time() - start))

### Regress out confounding factors (number of counts, mitochondrial gene expression)

We can now perform regression on the count matrix to correct for confounding factors -  for example purposes, we use the number of counts and the expression of mitochondrial genes (named starting with `mt-`).

We now calculate the total counts and the percentage of mitochondrial counts for each cell.

In [ ]:
%%time
# sparse_gpu_array = cp.sparse.csc_matrix(adata.X)
chunk_size = int(sparse_gpu_array.shape[0] / (n_workers*10))

dask_array = dask.array.from_array(
    sparse_gpu_array, 
    chunks=(chunk_size, -1),
    asarray=False)

# Filter highly variable genes
dask_array = dask_array[:, adata.var.highly_variable.values]
genes = genes[adata.var.highly_variable.values]

# TODO: File a bug/patch for transpose not working due to axes validation
# dask_array = dask.array.transpose(dask_array, axes=None)
sparse_gpu_array = dask_array.compute().tocsc()
sparse_gpu_array.shape

In [ ]:
%%time
mito_genes = genes.str.startswith(MITO_GENE_PREFIX)

n_counts = sparse_gpu_array.sum(axis=1)
percent_mito = (sparse_gpu_array[:,mito_genes].sum(axis=1) / n_counts).ravel()

n_counts = cp.array(n_counts).ravel()
percent_mito = cp.array(percent_mito).ravel()

And perform regression:

In [ ]:
%%time
dk_sparse_gpu_array = dask.array.from_array(sparse_gpu_array, chunks=chunk_size, asarray=False)

regressors = cp.ones((n_counts.shape[0]*3), dtype=cp.float32).reshape((n_counts.shape[0], 3), order="F")
regressors[:, 1] = n_counts
regressors[:, 2] = percent_mito

X = dask.array.from_array(regressors)
X = X.persist()


In [ ]:
%%time

# @delayed
# def todense(arr, i):
#     return arr[:, i]

# # dk_sparse_gpu_array[:, 0].compute()
# y = dask.array.from_delayed(sparse_gpu_array[:, 0],
#                             dtype=cp.float32,
#                             shape=(sparse_gpu_array.shape[0], 1))
# y.compute()
# sparse_gpu_array[:, 0]

# ld_delay = []
# results = []
# for i in range(sparse_gpu_array.shape[1]):
#     if i % 500 == 0:
#         print(i)

# #     y = dask.array.from_delayed(todense(sparse_gpu_array, i),
# #                                 dtype=cp.float32,
# #                                 shape=(sparse_gpu_array.shape[0], 1))

#     y = dask.array.from_delayed(todense(dk_sparse_gpu_array, i),
#                                 dtype=cp.float32,
#                                 shape=(sparse_gpu_array.shape[0], 1))
#     print(y.compute())
#     print(X.compute())

    # TODO: Following works but does not use all available GPUs
#     y = dask.array.from_array(sparse_gpu_array[:, i].todense())
    
#     lr1 = distLR(client=client, fit_intercept=False).fit(X, y).predict(X)
#     ld_delay.append(lr1)

# print('Stacking arrays...')
# ld_delay_array = dask.array.concatenate(ld_delay, allow_unknown_chunksizes=True)
# print('Computing...')
# results = ld_delay_array.compute()

# results

In [ ]:
%%time
# X = X.compute()
# chunk_size = sparse_gpu_array.shape[1] // (n_workers*2)

# from cuml.linear_model import LinearRegression

# @delayed
# def regress(X, y, i):
#     lr = LinearRegression(fit_intercept=False, output_type="cupy")
#     lr.fit(X, y, convert_dtype=True)
#     return (y.reshape(y.shape[0],) - lr.predict(X).reshape(y.shape[0])).get()

#     #TODO: Revisit the following code to distribute each LinearRegression distributed.
#     #      Current code fails due to client unavailability. Client cannot be serialized too.
#     # lr = distLR(fit_intercept=False)
#     # lr.fit(X, y)
#     # return y.reshape(y.shape[0],) - lr.predict(X).reshape(y.shape[0])

# print('Creating dask arrays for LinearRegression...')
# ld_delay = []
# for i in range(sparse_gpu_array.shape[1]):
#     tmp = time.time()
#     if i % 500 == 0:
#         print(i)

#     y = sparse_gpu_array[:, i].todense()
#     result = regress(X, y, i)
    
#     ld_delay.append(
#         dask.array.from_delayed(
#             regress(X, y, i),
#             dtype=cp.float32, 
#             shape=(sparse_gpu_array.shape[0], 1)))

# print('Stacking arrays...')
# ld_delay_array = dask.array.vstack(ld_delay)
# print('Computing...')
# result = ld_delay_array.compute(client=client)
# result
sparse_gpu_array = rapids_scanpy_funcs.regress_out(sparse_gpu_array, n_counts, percent_mito)
sparse_gpu_array.shape

### Scale

Finally, we scale the count matrix to obtain a z-score and apply a cutoff value of 10 standard deviations, obtaining the preprocessed count matrix.

In [ ]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.scale(sparse_gpu_array, max_value=10)

In [ ]:
preprocess_time = time.time()
print("Total Preprocessing time: %s" % (preprocess_time-preprocess_start))

## Cluster & Visualize

We store the preprocessed count matrix as an AnnData object, which is currently in host memory. We also add the expression levels of the marker genes as observations to the annData object.

In [ ]:
%%time
# genes = adata.var_names
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = genes

gene_filter_cols = genes[genes.isin(markers)]
# marker_genes_raw = zip(gene_filter_cols.index.to_arrow().to_pylist(), gene_filter_cols.values_host, )


# for name, data in marker_genes_raw.items():
#     adata.obs[name] = data.get()
    
adata.shape

In [ ]:
%%time
sparse_gpu_array.columns


# genes = cudf.Series(adata.var_names)
# del sparse_gpu_array

### Reduce

We use PCA to reduce the dimensionality of the matrix to its top 50 principal components. Here, we use Dask to parallelize across multiple GPUs.

In [ ]:
%%time
dask_array = dask.array.from_array(
    sparse_gpu_array, 
    chunks=(chunk_size, -1),
    asarray=False)

dask_array

The PCA multi-GPU time includes initial data transfer, which is about 20gb. 

In [ ]:
pca = PCA(n_components=50)
dask_array = pca.fit_transform(dask_array)
adata.obsm["X_pca"] = dask_array.compute().get()

### t-SNE + k-Means

In [ ]:
%%time
adata.obsm['X_tsne'] = TSNE().fit_transform(adata.obsm["X_pca"][:,:tsne_n_pcs])

We cluster the cells using k-means on the principal components. For example purposes, we set k=35.

In [ ]:
%%time
# K-means
dask_kmeans_output = KMeans(n_clusters=k).fit_predict(dask_array)
adata.obs['kmeans'] = dask_kmeans_output.compute().get().astype(str)

We visualize the cells using t-SNE and label cells by color according to the k-means clustering.

In [ ]:
# sc.pl.tsne(adata, color=["kmeans"])
x_embedded = TSNE().fit_transform(sparse_gpu_array)
x_embedded

In [ ]:
x_embedded.shape

We label the cells using the `Stmn2` and `Hes1` marker genes, for neuronal and glial cells respectively. These visualizations show us the separation of neuronal and glial cells on the t-SNE plot.

In [ ]:
%%time
sc.pl.tsne(adata, color=["Stmn2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.tsne(adata, color=["Hes1_raw"], color_map="Blues", vmax=1, vmin=-0.05)

### UMAP + Graph clustering

We can also visualize the cells using the UMAP algorithm in Rapids. Before UMAP, we need to construct a k-nearest neighbors graph in which each cell is connected to its nearest neighbors. This can be done conveniently using rapids functionality already integrated into Scanpy.

Note that Scanpy uses an approximation to the nearest neighbors on the CPU while the GPU version performs an exact search. While both methods are known to yield useful results, some differences in the resulting visualization and clusters can be observed.

In [ ]:
%%time
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')

The UMAP function from Rapids is also integrated into Scanpy.

In [ ]:
%%time
local_model = UMAP(n_epochs=1000, min_dist=umap_min_dist, spread=umap_spread)
local_model.fit(adata.obsm["X_pca"][:350000,:])

In [ ]:
%%time
dist_embeddings = distUMAP(local_model).transform(dask_array)
adata.obsm["X_umap"] = dist_embeddings.compute().get()

Finally, we use the Louvain algorithm for graph-based clustering, once again using the `rapids` option in Scanpy.

In [ ]:
%%time
sc.tl.louvain(adata, flavor='rapids')

We plot the cells using the UMAP visualization, and using the Louvain clusters as labels.

In [ ]:
%%time
sc.pl.umap(adata, color=["louvain"])

And also visualize the cells labeled by expression of the `Stmn2` and `Hes1` marker genes, for neuronal and glial cells respectively.

In [ ]:
%%time
sc.pl.umap(adata, color=["Stmn2_raw"], color_map="Blues", vmax=1, vmin=-0.05)
sc.pl.umap(adata, color=["Hes1_raw"], color_map="Blues", vmax=1, vmin=-0.05)

In [ ]:
cluster_time = time.time()
print("Total cluster time : %s" % (cluster_time-cluster_start))

## Create Zoomed View

The speedup offered by Rapids makes it easy to interactively re-analyze subsets of cells. To illustrate this, we select glial cells (Hes1+) from the dataset.

In [ ]:
reanalysis_start = time.time()

In [ ]:
%%time
hes1_cells = marker_genes_raw["Hes1_raw"] > 0.0
adata = adata[hes1_cells.get(),:]
adata.shape

We can repeat the dimension reduction, clustering and visualization using this subset of cells in seconds. 

Finally, we visualize the selected neuronal cells labeled by their new clusters, and by the expression of `Olig1`, a marker gene for oligodendrocytes.

In [ ]:
del dask_array

In [ ]:
%%time

chunk_size = int(sparse_gpu_array.shape[0] / (n_workers*10))
dask_array = dask.array.from_array(cp.asarray(adata.X), 
                                   chunks=(chunk_size, -1))
dask_array = dask_array.persist()

client.rebalance()

In [ ]:
%%time
dask_array = pca.fit_transform(dask_array)
adata.obsm['X_pca'] = dask_array.compute().get()

In [ ]:
%%time
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')
sc.tl.umap(adata, min_dist=umap_min_dist, spread=umap_spread, method='rapids')
sc.tl.louvain(adata, flavor='rapids')

sc.pl.umap(adata, color=["louvain"])
sc.pl.umap(adata, color=["Olig1_raw"], color_map="Blues", vmax=1, vmin=-0.05)

In [ ]:
reanalysis_time = time.time()
print("Total reanalysis time : %s" % (reanalysis_time-reanalysis_start))

In [ ]:
client.close()

In [ ]:
print("Full time: %s" % (time.time() - start))